# Ferramentas integradas: o processo de RAG está embutido dentro da ferramenta e podemos chamar o código de forma mais simples.


---



* Primeiro exemplo: Langchain integrado ao PubMed




Importando pacotes necessários

In [ ]:
!pip install xmltodict

In [ ]:
!pip install langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.9/393.9 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.9/148.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
from langchain_community.tools.pubmed.tool import PubmedQueryRun



In [ ]:
tool = PubmedQueryRun()

In [ ]:
print(tool.invoke("What causes lung cancer?"))

Published: 2023-10-20
Title: Liquid biopsy for early detection of lung cancer.
Copyright Information: © 2023 The Author.
Summary::
Lung cancer is the leading cause of cancer-related mortality worldwide. Early cancer detection plays an important role in improving treatment success and patient prognosis. In the past decade, liquid biopsy became an important tool for cancer diagnosis, as well as for treatment selection and response monitoring. Liquid biopsy is a broad term that defines a non-invasive test done on a sample of blood or other body fluid to look for cancer cells or other analytes that can include DNA, RNA, or other molecules released by tumor cells. Liquid biopsies mainly include circulating tumor DNA, circulating RNA, microRNA, proteins, circulating tumor cells, exosomes, and tumor-educated platelets. This review summarizes the progress and clinical application potential of liquid biopsy for early detection of lung cancer.

Published: 2023-12-08
Title: Asian, regional, and n

In [ ]:
print(tool.invoke("What are the advances of the last five years in drug discovery with artificial intelligence? "))

No good PubMed Result was found


# Integração dos componentes


---



Objetivo:


*   Carregar um dataset de apoio contendo uma cariedade de textos de bio
*   Dada uma pergunta, identificar no dataset de apoio informações que possam ser usadas como contexto pelo modelo generativo
*  Dada uma pergunta, gerar texto usando LLM e o contexto selecionado nas etapas anteriores

----





Instalando alguns pacotes do langchain e HuggingFace

O LangChain é uma estrutura de código aberto projetada para construir aplicações que utilizam grandes modelos de linguagem (LLMs)

In [ ]:
!pip install -U langchain-community
!pip install langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.5/391.5 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.9/148.9 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6

In [ ]:
!pip install transformers torch datasets
!pip install sentence-transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

Carregando os dados usando a biblioteca pandas

In [ ]:
import pandas as pd

df = pd.read_json("hf://datasets/botbot-ai/biology-ptbr/translated_dataset.json", lines=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Vamos ver os títulos das nossas colunas

In [ ]:
df.columns

Index(['role_1', 'topic;', 'sub_topic', 'message_1', 'message_2'], dtype='object')

Vamos ver uma amostra dos dados

In [ ]:
df.head(5)

,role_1,topic;,sub_topic,message_1,message_2
0,Biologist_RoleType.ASSISTANT,Taxonomia,Identificação de novas espécies,Quais são as técnicas comuns usadas na identif...,Este método envolve um estudo genético: Este é...
1,Biologist_RoleType.ASSISTANT,Taxonomia,Identificação de novas espécies,Como pode a sequenciação do ADN ser utilizada ...,O sequenciamento de DNA pode ser usado para id...
2,Biologist_RoleType.ASSISTANT,Taxonomia,Identificação de novas espécies,"""Como você pode identificar e classificar uma ...",Para identificar e classificar uma espécie rec...
3,Biologist_RoleType.ASSISTANT,Taxonomia,Identificação de novas espécies,Identificar as principais diferenças entre dua...,Para identificar as principais diferenças entr...
4,Biologist_RoleType.ASSISTANT,Taxonomia,Identificação de novas espécies,Como pode a análise do ADN ser usada na identi...,A análise de DNA pode ser usada na identificaç...


Quantas amostras temos nesse dataset?

In [ ]:
df.shape

(20000, 5)

Para agilizar, selecionamos apenas 100 amostras e reduzimos o número de colunas

In [ ]:
df["message_1"][0]

'Quais são as técnicas comuns usadas na identificação de uma nova espécie, e como os cientistas podem categorizá-la com precisão dentro do sistema de taxonomia existente?'

In [ ]:
df["message_2"][0]

'Este método envolve um estudo genético: Este é o método tradicional de identificar uma nova espécie com base em suas características físicas. Este estudo inclui técnicas epitômicas. Este estudo também pode ajudar os cientistas a identificar uma nova espécie com base em suas características físicas. Este estudo inclui técnicas biológicas. Este estudo inclui técnicas de classificação biológica. Este estudo inclui técnicas de classificação biológica. Este estudo inclui técnicas de classificação biológica. Este estudo pode ajudar os cientistas a entender melhor a estrutura e a distribuição de um novo organismo, incluindo a sua estrutura esquelética, muscular e reprodutiva. Ao examinar a sua posição no contexto de uma nova espécie, pode ajudar os cientistas a determinar de forma mais precisa e mais precisa as diferenças entre a sua posição no ecossistema, a sua posição no nicho, a sua distribuição e a sua distribuição.'

In [ ]:
df = df.head(100)
df = df[["topic;", "sub_topic", "message_2"]]

In [ ]:
set(df['topic;'])

{'Taxonomia'}

In [ ]:
set(df['sub_topic'])

{'Classificação dos organismos com base nas suas características físicas',
 'Determinação das relações evolutivas entre diferentes espécies',
 'Identificação de novas espécies',
 'Identificação de subespécies dentro de uma espécie'}

In [ ]:
df["message_2"].head(5)

,message_2
0,Este método envolve um estudo genético: Este é...
1,O sequenciamento de DNA pode ser usado para id...
2,Para identificar e classificar uma espécie rec...
3,Para identificar as principais diferenças entr...
4,A análise de DNA pode ser usada na identificaç...


Vamos ler todo o texto da coluna "message_2" e armazenar em uma lista (data)

In [ ]:
data = df["message_2"].to_list()
print(data[2])
print(len(data))

Para identificar e classificar uma espécie recém-descoberta no reino Animalia usando taxonomia, você pode seguir estas etapas: 1. Observe as características físicas do organismo: Comece examinando a morfologia do organismo, incluindo seu tamanho, forma, cor e estrutura corporal. Observe quaisquer características únicas que possam ajudar a diferenciá-lo de espécies conhecidas. 2. Analise o habitat e comportamento do organismo: Observe o habitat natural do organismo, seus hábitos alimentares, comportamento reprodutivo e qualquer outra informação ecológica relevante. 3. Colete amostras para análise genética: Obtenha amostras de tecido do organismo para extração e sequenciamento de DNA. Isso ajudará você a comparar seu material genético com o de espécies conhecidas e determinar suas relações evolutivas. 4. Consulte chaves e chaves taxonômicas: Use chaves taxonômicas e materiais de referência para comparar as características do organismo com as dessas espécies conhecidas. Isso permitirá que

Lembram dos embeddings?

Para a seleção do contexto que será entregue a LLM, vamos fazer uma busca semântica. O primeiro passo vai ser transformar cada amostra em um embedding, que é a representação preferida dos modelos atuais!

Minha primeira tentativa foi com o modelo `neuralmind/bert-base-portuguese-cased`. Esse é um modelo BERT que passou por um treinamento continuado com dados em Português. Mas ele não "entende" tão bem a linguagem do nosso domínio. Partimos para o modelo biobertpt-all.

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

# Define the path to the pre-trained model you want to use
#modelPath = "neuralmind/bert-base-portuguese-cased"
modelPath = "pucpr/biobertpt-all"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
# from langchain.text_splitter import TokenTextSplitter,CharacterTextSplitter
# text_splitter = TokenTextSplitter(chunk_size=128, chunk_overlap=50)
# chunks = text_splitter.split_documents(data)

Os embedding precisam ser armazenados em algum lugar: Chromadb

In [ ]:
from langchain.vectorstores import Chroma

In [ ]:
persist_directory = "chroma"
metadata = df.to_dict('records')


In [ ]:
vectordb = Chroma.from_texts(
    texts=data,
    metadatas=metadata,
    collection_name = 'teste',
    embedding=embeddings,
    persist_directory=persist_directory
)
print(f"Documents Loaded: {vectordb._collection.count()}")

Documents Loaded: 100


Antes de usar a LLM para gerar texto, vamos dar uma olhada no retorno da nossa busca semântica.

Dada a query "Como o estudo do DNA pode ajudar na descoberta de novas espécies?", nossa busca semântica retorna os top 3 melhores trechos armazenados no nosso banco.


In [ ]:
query = "Como o estudo do DNA pode ajudar na descoberta de novas espécies?"
k = 3
results = vectordb.similarity_search_with_relevance_scores(query, k=k)
results

/usr/local/lib/python3.10/dist-packages/langchain_core/vectorstores/base.py:559: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={'message_2': 'Estes métodos permitem uma compreensão mais precisa e detalhada dos limites das espécies e das relações evolutivas envolvidas. Aqui estão algumas maneiras moleculares e métodos genéticos podem ser usados para identificar novas espécies e diferenciá-las de espécies estreitamente relacionadas no mesmo gênero: 1. codificação de DNA: Esta técnica envolve a sequenciação de uma região padronizada do DNA, geralmente o mitocôndrio citocromo case oxidação subunidade I (COI) gene em animais, e compará-lo com uma base de dados de referência de espécies conhecidas. Se a sequência é significativamente diferente de qualquer espécie conhecida, pode representar uma nova espécie. A codificação de DNA também pode ser usada para diferenciar estreitamente relacionados, comparando suas relações genéticas. 2.', 'sub_topic': 'Identifica

[(Document(metadata={'message_2': 'Estes métodos permitem uma compreensão mais precisa e detalhada dos limites das espécies e das relações evolutivas envolvidas. Aqui estão algumas maneiras moleculares e métodos genéticos podem ser usados para identificar novas espécies e diferenciá-las de espécies estreitamente relacionadas no mesmo gênero: 1. codificação de DNA: Esta técnica envolve a sequenciação de uma região padronizada do DNA, geralmente o mitocôndrio citocromo case oxidação subunidade I (COI) gene em animais, e compará-lo com uma base de dados de referência de espécies conhecidas. Se a sequência é significativamente diferente de qualquer espécie conhecida, pode representar uma nova espécie. A codificação de DNA também pode ser usada para diferenciar estreitamente relacionados, comparando suas relações genéticas. 2.', 'sub_topic': 'Identificação de novas espécies', 'topic;': 'Taxonomia'}, page_content='Estes métodos permitem uma compreensão mais precisa e detalhada dos limites da

Mas o que nós queremos mesmo é usar esse contexto na nossa geração de texto, para nos auxiliar a responder perguntas nesse nosso novo domínio.

Como fizemos em outros exercícios, vamos usar o HuggingFace. Aqui estamos carregando o modelo Phi3, vocês podem experimentar com outros modelos.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline

model_name = "microsoft/Phi-3-mini-128k-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=128)
llm = HuggingFacePipeline(
    pipeline = pipe,
    model_kwargs={"temperature": 0.5, "max_length": 512}
)

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


Hora de elaborar um prompt. Nossa prompt vai receber 2 valores: o contexto recuperado via busca semântica e a pergunta que queremos que o modelo responda.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

PROMPT_TEMPLATE = """Responda a pergunta usando apenas o contexto a seguir:
{context}

Question: {question}
"""
PROMPT = PromptTemplate.from_template(PROMPT_TEMPLATE)

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(k=3),
    chain_type_kwargs={"prompt": PROMPT},
    return_source_documents=True
)

Hora de testar nosso modelo com RAG!

In [ ]:
query = "Como o estudo do DNA pode ajudar na descoberta de novas espécies?"
result = qa_chain({"query": query})

In [ ]:
print(result['result'].strip())

Responda a pergunta usando apenas o contexto a seguir:
Estes métodos permitem uma compreensão mais precisa e detalhada dos limites das espécies e das relações evolutivas envolvidas. Aqui estão algumas maneiras moleculares e métodos genéticos podem ser usados para identificar novas espécies e diferenciá-las de espécies estreitamente relacionadas no mesmo gênero: 1. codificação de DNA: Esta técnica envolve a sequenciação de uma região padronizada do DNA, geralmente o mitocôndrio citocromo case oxidação subunidade I (COI) gene em animais, e compará-lo com uma base de dados de referência de espécies conhecidas. Se a sequência é significativamente diferente de qualquer espécie conhecida, pode representar uma nova espécie. A codificação de DNA também pode ser usada para diferenciar estreitamente relacionados, comparando suas relações genéticas. 2.

O sequenciamento de DNA pode ser utilizado para identificar e classificar novas espécies dentro do ramo taxonômico da biologia através das seguin